<a href="https://colab.research.google.com/github/rpalsaxena/Daily_learning/blob/main/pythia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets loralib sentencepiece
!pip uninstall transformers
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.2 MB/s eta 0:00:00
     ━━

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

model_chkpt = "EleutherAI/pythia-70m"
tokenizer = AutoTokenizer.from_pretrained(model_chkpt)

In [7]:
data_stack = load_dataset("flax-sentence-embeddings/stackexchange_titlebody_best_voted_answer_jsonl", 'coffee')

def generate_prompt(data_point):
    # taken from https://github.com/tloen/alpaca-lora
    if data_point["title_body"]:
        return f"""Below is the context that mentions questions related to coffee that provides further context.
         Try to answer the question.

          ### Title:
          {data_point["title_body"]}

          ### Upvoted Answer:
          {data_point["upvoted_answer"]}

          """
    else:
        return f"""Below is the context that mentions questions related to coffee that provides further context.
         Try to answer the question.

          ### Response:
          {data_point["upvoted_answer"]}"""

data = data_stack.map(lambda data_point: {"prompt": tokenizer(generate_prompt(data_point))})
data

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['title_body', 'upvoted_answer', 'prompt'],
        num_rows: 1188
    })
})

In [9]:
train_data = data['train']['prompt']

In [12]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
import transformers
from transformers import AutoTokenizer, AutoModel


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [13]:
# Settings for A100 - For 3090
MICRO_BATCH_SIZE = 8  # change to 4 for 3090
BATCH_SIZE = 16
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 3 # paper uses 3
LEARNING_RATE = 0.0001
CUTOFF_LEN = 256

In [21]:
tokenizer = AutoTokenizer.from_pretrained(model_chkpt,
                                          add_eos_token=True,
                                          )

tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [15]:
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model

model = AutoModelForCausalLM.from_pretrained(model_chkpt,
                                  device_map="auto",
                                  ).to("cuda")

model = prepare_model_for_int8_training(model, use_gradient_checkpointing=True)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [22]:
training_args = transformers.TrainingArguments(output_dir="test_trainer",
                                               num_train_epochs=EPOCHS,
                                                learning_rate=LEARNING_RATE,
                                                )

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    args= training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False

In [23]:
trainer.train()

You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1537 in train                    │
│                                                                                                  │
│   1534 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1535 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1536 │   │   )                                                                                 │
│ ❱ 1537 │   │   return inner_training_loop(                                                       │
│   1538 │   │   │   args=args,                                                                    │
│   1539 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1540 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1802 in _inner_training_loop     │
│                                                                                                  │
│   1799 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1800 │   │   │   │                                                                             │
│   1801 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1802 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1803 │   │   │   │                                                                             │
│   1804 │   │   │   │   if (                                                                      │
│   1805 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2658 in training_step            │
│                                                                                                  │
│   2655 │   │   │   with amp.scale_loss(loss, self.optimizer) as scaled_loss:                     │
│   2656 │   │   │   │   scaled_loss.backward()                                                    │
│   2657 │   │   else:                                                                             │
│ ❱ 2658 │   │   │   self.accelerator.backward(loss)                                               │
│   2659 │   │                                                                                     │
│   2660 │   │   return loss.detach() / self.args.gradient_accumulation_steps                      │
│   2661                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:1821 in backward               │
│                                                                                                  │
│   1818 │   │   elif self.scaler is not None:                                                     │
│   1819 │   │   │   self.scaler.scale(loss).backward(**kwargs)                                    │
│   1820 │   │   else:                                                                             │
│ ❱ 1821 │   │   │   loss.backward(**kwargs)                                                       │
│   1822 │                                                   

In [ ]:
model.save_pretrained("python-fine-tuned")